<h1> CS753 Assignment 3 Graph Mining Algorithms</h1>


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import time

In [3]:
# Load dataset
data = np.loadtxt(r'....web-Google.txt',dtype= int)
print ("total number of links: ", len(data))

total number of links:  5105039


<h3>Task 1: Implementation of Power Iteration Algorithm.</h3> 

<h4>(a) Implement the power iteration algorithm in matrix form without teleport (on slides page 20): Let stop criteria be ||r (t+1) − r (t) ||1 < 0.02. Spider traps and dead ends are not considered in this task [30 pts].</h4>
    
<h4> <font color='blue'>Total number of unique source nodes is 739454. Because of big data nature, we use sparsematrix to store the adjacency matrix. In order to put source nodes as column and destinations as rows, I did a transpose before matrix multiplication with initial votes. The convergence happens when newrank - oldrank smaller than a threshold 0.02. Results show the column sum of sparseMatrix </font> </h4>

In [4]:
# get counts of repetitive source nodes

source,ind,counts = np.unique(data[:,0],return_inverse=True, return_counts=True)
print("Total unique source nodes: ", len(source))
# get initial votes for each node

votes = 1/counts[ind]        

# size of sparse matrix
SM_size = max(max(data[:,0]),max(data[:,1]))+1
# creating sparse matrix
sparseMatrix = csr_matrix((votes, (data[:,0], data[:,1])), 
                          shape = (SM_size, SM_size))

# initialize ranks
rank = np.ones((SM_size,1))/SM_size
# Power iteration algorithm
t1 = time.time()
flag = True
n=0
while flag:
    n += 1
    sparseMatrix_t = sparseMatrix.transpose() # transpose sparseMatrix to destination as rows and source as columns
    newrank = sparseMatrix_t.dot(rank)
    #Stop condition
    if np.linalg.norm(rank-newrank,ord=1)<0.02:
        flag = False        
    rank = newrank 
t2 = time.time()

## column sum of sparsematrix

colsum = sparseMatrix_t.sum(axis=0)

print("column sum of sparse matrix :", colsum)


Total unique source nodes:  739454
column sum of sparse matrix : [[1. 1. 1. ... 1. 1. 1.]]


<h4>(b) Calculate the rank score for all the nodes and report: (1) The running time of your power iteration algorithm; (2) the number of iterations needed to stop; (3) the IDs and scores of the top-10 ranked nodes [10 pts].</h4>

In [5]:
print("Total running time is ",round((t2-t1)*1000), "ms")
print("Total number of iterations till convergence : ", n)

top10_rank = np.sort(np.squeeze(rank))[::-1]
top10_node = np.argsort(np.squeeze(rank))[::-1]
print("IDs and scores of the top-10 ranked nodes: ", list(zip(top10_node[0:10],top10_rank[0:10])))

Total running time is  1748 ms
Total number of iterations till convergence :  62
IDs and scores of the top-10 ranked nodes:  [(6116, 0.0006177867154815842), (69056, 0.0006065543233976054), (69055, 0.0006065543233976054), (69057, 0.0006065543233976054), (31563, 0.00038756780164943863), (572672, 0.0003482264479290426), (572673, 0.00030958996008457015), (60232, 0.00027105054535807397), (572674, 0.0002702807179711503), (33676, 0.00025978830964025945)]


<h3>2.Task 2: Exploiting dead-ends:</h3>

<h4>Before extending your codes to support dead ends, let’s first do some analysis on your current implementation in Task1. Report the leaked PageRank score in each iteration of task 1 and explain the phenomenon you observe from the above experiments.</h4>

<h4><font color='blue'> According to the result shown underlying, rating scores of dead-end nodes are continuously reducing over 20 iterations, indicating the pageRank scores are leaked through the dead-end nodes. In addition, the indices of dead-end nodes and total number of dead-end nodes are reported. The sum of final total rankings after convergence is 0.193, << 1,giving further evidence of pageRank leaking.  </font></h4>


In [ ]:
nodes_deadend = np.where(colsum==0)[1]
print("indices of dead-end nodes: ", nodes_deadend)
print("total number of dead-end nodes: ", len(nodes_deadend))

# get counts of repetitive source nodes

source,ind,counts = np.unique(data[:,0],return_inverse=True, return_counts=True)

# get initial votes for each node

votes = 1/counts[ind]        

# size of sparse matrix
SM_size = max(max(data[:,0]),max(data[:,1]))+1
# creating sparse matrix
sparseMatrix = csr_matrix((votes, (data[:,0], data[:,1])),shape = (SM_size, SM_size))

# initialize ranks
rank = np.ones((SM_size,1))/SM_size
    
flag = True
n=0
epsilon = 0.02
while flag:
    print()
    n += 1
    sparseMatrix_t = sparseMatrix.transpose()
    newrank = sparseMatrix_t.dot(rank)
    leakedrank= 1-sum(newrank)
    while n <=20:
        print("iteration n: ",n, "leaded ranking: ", leakedrank)
    #Stop condition
    if np.linalg.norm(rank-newrank,ord=1)<epsilon:
        flag = False
    rank = newrank
print("Sum of total rankings after convergence : ", sum(rank))

indices of dead-end nodes:  [    17     33     35 ... 875702 875706 875707]
total number of dead-end nodes:  136259


<h3>3. Task 3: Implementation of Teleport.</h3>

<h4>(a) Extend your PageRank code to handle both spider traps and dead ends using the idea of teleport. Let β = 0.9 by default and the stop criteria be ||r(t+1) −r(t)||1 <0.02. </h4>

<h4> <font color='blue'>Firstly all rank score would muptiply beta value, 0.9. The value of (1 - sum of all rank scores) is calculated and evenly added back into the new ranking scores. The purpose of doing it is to teleport the leaking scores back to the total network and ensure the column sum is still equal to 1 as shown underlying.</font></h4>

In [46]:
# get counts of repetitive source nodes
source,ind,counts = np.unique(data[:,0],return_inverse=True, return_counts=True)

# get initial votes for each node

votes = 1/counts[ind]        

# size of sparse matrix
SM_size = max(max(data[:,0]),max(data[:,1]))+1
# creating sparse matrix
sparseMatrix = csr_matrix((votes, (data[:,0], data[:,1])),shape = (SM_size, SM_size))
# Power iteration algorithm with teleportation
t = time.time()
def pRank_tele(beta):
    # initialize ranks
    rank = np.ones((SM_size,1))/SM_size
    epsilon = 0.02
    flag = True
    n=0
    while flag:
        print()
        n += 1
        sparseMatrix_t = sparseMatrix.transpose() # transpose sparseMatrix to destination as rows and source as columns
        newrank1 = (sparseMatrix_t.dot(rank))*beta
        # teleport leaked pagerank
        newrank2 = newrank1 + (1-newrank1.sum())/SM_size
        #Stop condition
        if np.linalg.norm(rank-newrank2,ord=1)<epsilon:
            flag = False        
        rank = newrank2
    return (n,rank)
teleRank = pRank_tele(0.9)
print(" Sum of total ranking after teleport : ", sum(teleRank[1]))
t_end = time.time()  












 Sum of total ranking after teleport :  [1.]


<h4>(b) Run your code on the Google web data and report: (1) the running time; (2) the
number of iterations needed to stop; (3) the IDs and scores of the top-10 ranked
nodes [10 pts].</h4>

In [43]:
print("Total running time is: ",round((t2-t1)*1000), "ms")
print("Total number of iterations till convergence : ", teleRank[0])
top_rank = np.sort(np.squeeze(teleRank[1]))[::-1]
top_node = np.argsort(np.squeeze(teleRank[1]))[::-1]
print("IDs and scores of the top-10 ranked nodes after teleport: ", list(zip(top_node[0:10],top_rank[0:10])))


Total running time is:  1791 ms
Total number of iterations till convergence :  11
IDs and scores of the top-10 ranked nodes after teleport:  [(2138, 0.0010085161582659716), (115, 0.0009705537443270651), (3178, 0.0009380557550726863), (2560, 0.0009314724701097537), (1950, 0.0008509553126824594), (1181, 0.0008113770426201482), (903, 0.0007834380557670724), (1611, 0.00075737276793708), (3150, 0.0007537883564045684), (3180, 0.0007401293020022142)]


<h4>(c) By varying the teleport probability β in [1, 0.9, 0.8, 0.7, 0.6, 0.5], report the number of iterations needed to stop for each β. Explain your findings from this
experiment [10 pts].</h4>

<h4> <font color='blue'> From results shown underlying, we can see with decreasing beta value, the total number of iterations is also reduced. The total iterations at beta =1 is 90, which is different from the result obtained previously 62. The reason is when without teleport, it is not actual convergence. The total sum reduced (0.19 << 1) due to ranking score leakage. Therefore when the difference of ranking below threshold (epsilon 0.02), it reached so-called convergence due to the diminishing scores. When teleport is implemented, the convergence is the real convergence because the total sum of the final ranking score is 1, there is no leakage of scores.</font> </h4>

In [47]:
for beta in  [1, 0.9, 0.8, 0.7, 0.6, 0.5]:
    N_iter = pRank_tele(beta)[0]
    print("when beta =", beta,"Number of iterations :", N_iter)




























































































when beta = 1 Number of iterations : 90











when beta = 0.9 Number of iterations : 11







when beta = 0.8 Number of iterations : 7






when beta = 0.7 Number of iterations : 6





when beta = 0.6 Number of iterations : 5




when beta = 0.5 Number of iterations : 4
